In [1]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head() # 데이터 프레임의 첫번째 행부터 5개 추출

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
# pandas unique: 명시된 컬럼에 중복 제거
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [3]:
# Species: 타겟 데이터(정답 데이터), 그외 - 학습 데이터
import numpy as np
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [4]:
fish_target = fish['Species'].to_numpy() # 타겟 데이터(정답 데이터)

In [5]:
from sklearn.model_selection import train_test_split # 학습 세트,테스트 세트 분리

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target)

In [6]:
# 데이터 전처리 - 표준 점수 변환
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input) # 훈련 세트 훈련 데이터를 표준 점수로 변환
test_scaled = ss.transform(test_input) # 테스트 세트 데이터를 표준 점수로 변환
print(train_scaled[:5])

[[-0.66934297 -0.49636322 -0.58774438 -0.69194122 -0.44331502]
 [-1.15297227 -1.7387142  -1.7601056  -1.67239093 -1.97819212]
 [-1.1421042  -1.61750922 -1.6577566  -1.59127542 -1.98333388]
 [-0.42783021 -0.04184457 -0.1783484  -0.3304767  -0.14283577]
 [-0.94708264 -0.98118312 -1.10879381 -0.94356768 -0.82054439]]


In [7]:
# k- 최근접 이웃 분류 - 분류, 확율 예측
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target) # 학습 / 타겟 데이터와 정답 데이터를 학습 시킴

print(kn.score(train_scaled, train_target)) # 훈련 세트 정확도
print(kn.score(test_scaled, test_target)) # 테스트 세트 정확도

0.8907563025210085
0.85


In [8]:
# 클래스 목록(분류 항목) - 알파벳 순서대로 표시
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [9]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Perch' 'Pike' 'Bream' 'Perch']


In [10]:
# 분류 기준 확률 kn.predict_proba(..)
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.    0.667 0.    0.333 0.    0.   ]
 [0.    0.    0.333 0.    0.333 0.333 0.   ]
 [0.    0.    0.    1.    0.    0.    0.   ]
 [1.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    1.    0.    0.    0.    0.   ]]


In [11]:
distances, indexes = kn.kneighbors(test_scaled[2:3])
print(train_target[indexes]) # 확률로 보기에는 조금 어색 -< 로지스틱 회귀

[['Pike' 'Pike' 'Pike']]


In [12]:
# 로지스틱 회귀 - 이진 분류
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_brean_smelt = train_target[bream_smelt_indexes]
print(target_brean_smelt[:5])

['Smelt' 'Smelt' 'Smelt' 'Bream' 'Bream']


In [13]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_brean_smelt) # 훈련

LogisticRegression()

In [14]:
# 확률 predict_proba
print(np.round(lr.predict_proba(train_bream_smelt[:5]), decimals=3))

[[0.034 0.966]
 [0.042 0.958]
 [0.034 0.966]
 [0.989 0.011]
 [0.96  0.04 ]]


In [15]:
print(lr.classes_)

['Bream' 'Smelt']


In [16]:
# 가중치(기울기)와 절편
print(lr.coef_,lr.intercept_)

[[-0.42323119 -0.613403   -0.69621143 -0.94415891 -0.76936334]] [-2.54273509]


In [18]:
# 학습을 통해서 구한 가중치와 저편으로 방정식의 결과 값을 구하는 함수 decision_function
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions) # 결과 값을 0~1 사이로 값으로 변환 -> 시그모이드 함수

[ 3.33812817  3.11529438  3.33971066 -4.4676207  -3.16583764]


In [21]:
# 시그모이드 함수로 값을 0~1 사이로 변환
from scipy.special import expit # 시그모이드 함수
print(np.round(expit(decisions), decimals=3))

[0.966 0.958 0.966 0.011 0.04 ]


In [23]:
# 로지스틱 회귀 - 다중 분류 / 확률 - 소프트맥스 함수
lr = LogisticRegression(max_iter=10000)
lr.fit(train_scaled, train_target) # 학습

print(lr.score(train_scaled, train_target)) # 학습 세트
print(lr.score(test_scaled, test_target)) # 테스트 세트

0.8067226890756303
0.85


In [25]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Bream' 'Perch']


In [27]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [29]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.014 0.038 0.629 0.025 0.236 0.004 0.054]
 [0.001 0.078 0.291 0.004 0.166 0.453 0.008]
 [0.016 0.001 0.237 0.655 0.057 0.    0.034]
 [0.921 0.008 0.026 0.005 0.007 0.    0.033]
 [0.056 0.    0.859 0.012 0.006 0.    0.067]]


In [31]:
print(lr.coef_,lr.intercept_)

[[-0.07700843 -0.09880984  0.74877219  2.92842381 -0.28181387]
 [-0.37040607 -0.80293733 -0.80600653  2.05759003 -0.83761981]
 [ 1.2430027   0.45094127 -1.2718084  -1.75848717  1.66037571]
 [-0.08561139  1.74343445  1.74606801 -1.28643831 -0.70511905]
 [-0.61111047 -0.43869442  0.38960205 -0.58569496  0.66783342]
 [-0.35615735 -0.84790019 -0.86374999 -1.33849125 -1.57293464]
 [ 0.25729101 -0.00603394  0.05712266 -0.01690215  1.06927825]] [ 0.769287   -0.02784538  2.21847739 -0.01467325  1.06742799 -4.5442001
  0.53152634]


In [34]:
# decision_function
decisions = lr.decision_function(test_scaled[:5])
print(np.round(decisions, decimals=3))

[[ -1.167  -0.17    2.637  -0.595   1.658  -2.542   0.18 ]
 [ -4.065   0.922   2.241  -2.063   1.676   2.683  -1.395]
 [ -0.052  -2.548   2.662   3.676   1.232  -5.678   0.709]
 [  5.254   0.528   1.706  -0.048   0.437  -9.792   1.915]
 [  2.901  -2.572   5.634   1.378   0.644 -11.072   3.087]]


In [38]:
from scipy.special import softmax

proba = softmax(decisions, axis=1) # 행별로 axis=1, 행 기준으로 보기 위해서 사용
print(np.round(proba, decimals=3))

[[0.014 0.038 0.629 0.025 0.236 0.004 0.054]
 [0.001 0.078 0.291 0.004 0.166 0.453 0.008]
 [0.016 0.001 0.237 0.655 0.057 0.    0.034]
 [0.921 0.008 0.026 0.005 0.007 0.    0.033]
 [0.056 0.    0.859 0.012 0.006 0.    0.067]]
